In [1]:
## IMPORTS
import os
import os.path
import struct
import numpy

## USER PARAMETERS
rootdir = os.path.abspath('/media/guille/DADES/DADES/David-cerditos/Datos_cerditos/000_CLEANED_PIGS/')

In [2]:
# Copyright (C) 2018 - Universitat Pompeu Fabra
# Author - Guillermo Jimenez-Perez <guillermo.jim.per@gmail.com>

# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.

# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

## GENERAL PARAMETERS
SETW_SEPARATION                       = 9

DICOM_HEADER_FIELD_PRECURSOR          = 4
DICOM_HEADER_FIELD_TYPE               = 4

FILE_HEADER_TOTAL_SIZE                = 276
FILE_HEADER_FIXED_BEGINNING           = 4

FILE_HEADER_FILE_SIZE                 = 4
FILE_HEADER_FILE_TYPE                 = 4
FILE_HEADER_ENDING_CODE               = 4

MPT_FILES_COMMON_HEADER_SIZE          = 12
MPT_FILES_NUMBER_OF_TRACKS_1          = 8
MPT_FILES_NUMBER_OF_TRACKS_2          = 9

MPT_FILE_TOTAL_SIZE                   = 5012
MPT_FILE_TRACK_HEADER                 = 8
MPT_FILE_TRACK_HEADER_TRACK_NUMBER_1  = 0
MPT_FILE_TRACK_HEADER_TRACK_NUMBER_2  = 1
MPT_FILE_ECG_TRACK_DATA               = 5000
MPT_FILE_TRACK_ENDING                 = 4

CATHETER_POSITION_HEADER              = 4
CATHETER_SEGMENTS                     = 251
CATHETER_POSITION_TRACK_SIZE          = CATHETER_SEGMENTS*32 # 251 segments of 32 bytes
CATHETER_POSITION_ENDING              = 4

TOTAL_SEGMENT_SIZE                    = 32
TIME_INDEX_SIGNED_LONG                = 4
TIME_INDEX_LONG                       = 4
X_POSITION                            = 4
Y_POSITION                            = 4
Z_POSITION                            = 4
ANGLE_ALPHA                           = 4
ANGLE_BETA                            = 4
ANGLE_THETA                           = 4

ECG_TRACK_DATA_POINTS                 = 2500
R_WAVE_POSITION                       = 2000
R_WAVE_POSITION_10_HZ                 = 200
LAT_CALCULATION_WINDOW                = 150
UNIPOLAR_TRACK                        = 19
BIPOLAR_TRACK                         = 20

# MAY CHANGE FROM FILE TO FILE!!!
TRACK_EQUIVALENCE = dict()

TRACK_EQUIVALENCE[1] = "I"
TRACK_EQUIVALENCE[2] = "II"
TRACK_EQUIVALENCE[3] = "III"
TRACK_EQUIVALENCE[4] = "aVR"
TRACK_EQUIVALENCE[5] = "aVL"
TRACK_EQUIVALENCE[6] = "aVF"
TRACK_EQUIVALENCE[7] = "V1"
TRACK_EQUIVALENCE[8] = "V2"
TRACK_EQUIVALENCE[9] = "V3"
TRACK_EQUIVALENCE[10] = "V4"
TRACK_EQUIVALENCE[11] = "V5"
TRACK_EQUIVALENCE[12] = "V6"
TRACK_EQUIVALENCE[20] = "M1"
TRACK_EQUIVALENCE[21] = "M2"
TRACK_EQUIVALENCE[22] = "M3"
TRACK_EQUIVALENCE[23] = "M4"
TRACK_EQUIVALENCE[40] = "R1"
TRACK_EQUIVALENCE[41] = "R2"
TRACK_EQUIVALENCE[60] = "M1-M2"
TRACK_EQUIVALENCE[61] = "M3-M4"
TRACK_EQUIVALENCE[80] = "R1-R2"
TRACK_EQUIVALENCE[81] = "R3-R4"

if not os.path.exists(os.path.join(rootdir,'OUTPUT')):
    os.makedirs(os.path.join(rootdir,'OUTPUT'))

# Iterate over the files in the folder
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        extension = os.path.splitext(file)
        
        # If the file is a .dicm file
        if extension[1] == '.dicm':
            # Read the binary file 
            with open(os.path.join(subdir, file), "rb") as dicmFile:
                binaryFile = dicmFile.read();
            
            pointLocations = []
            pointNumbers   = []
            searchPosition = 0
            
            # If there exists a .mpt file in the .dicm file:
            if binaryFile.find(b'.mpt') >= 0:
                while searchPosition < len(binaryFile):
                    # Find starting point of header file from the path of the .mpt file. 
                    # Drive letter is ommited
                    positionHeader = binaryFile.find(b':\\\\', searchPosition) - FILE_HEADER_FIXED_BEGINNING - 1
                    
                    # Find location of .mpt extension
                    positionMPT    = binaryFile.find(b'.mpt', positionHeader, positionHeader + FILE_HEADER_TOTAL_SIZE)
                    
                    # If the combination of the drive and the .mpt extension are found:
                    if (positionHeader >= 0) and (positionMPT >= 0): 
                        # Find the full filename of the MPT file referenced by the .dicm file
                        pointFileName  = binaryFile[positionHeader+FILE_HEADER_FIXED_BEGINNING:positionMPT+len('.mpt')]

                        # Store the point number referenced by the MPT file (e.g. P49.mpt) and its position
                        pointNumbers.append(int(os.path.splitext(pointFileName.split('\\')[-1])[0][1:]))
                        pointLocations.append(positionHeader)
                    
                    # If no more files (DRIVE:\\AAA\\BBB\CCC\\PXYZ.mpt) are found:
                    if positionHeader < 0:
                        # Finish the loop
                        searchPosition = len(binaryFile) + 1
                    else:
                        # Update search position for next .mpt file
                        searchPosition = positionHeader + FILE_HEADER_FIXED_BEGINNING + 2
                        
            # For each .mpt file located in the .dicm file:
            for i in range(0,len(pointLocations)):
                # Get the number of tracks the .mpt file contains. The information is encoded in the header 
                # of the MPT file, in the 8th and 9th bytes, as a little endian short:
                numberOfTracks = struct.unpack('<h',binaryFile[pointLocations[i]+FILE_HEADER_TOTAL_SIZE+MPT_FILES_NUMBER_OF_TRACKS_1:pointLocations[i]+FILE_HEADER_TOTAL_SIZE+MPT_FILES_NUMBER_OF_TRACKS_2+1])
                
                # Create a matrix for storing the point information
                ekgData        = numpy.zeros((numberOfTracks[0],int(MPT_FILE_ECG_TRACK_DATA/2)))
                latData        = numpy.zeros((numberOfTracks[0],1))
                catheter1Data  = numpy.zeros((6,CATHETER_SEGMENTS))
                catheter2Data  = numpy.zeros((6,CATHETER_SEGMENTS))

                trackCodes     = []
                
                for j in range(0, numberOfTracks[0]):
                    # For convenience, we store the individual track into a variable
                    trackInitialPos = pointLocations[i] + FILE_HEADER_TOTAL_SIZE + MPT_FILES_COMMON_HEADER_SIZE + MPT_FILE_TRACK_HEADER + j*(MPT_FILE_ECG_TRACK_DATA + MPT_FILE_TRACK_ENDING + MPT_FILE_TRACK_HEADER)
                    trackFinalPos   = pointLocations[i] + FILE_HEADER_TOTAL_SIZE + MPT_FILES_COMMON_HEADER_SIZE + MPT_FILE_TRACK_HEADER + j*(MPT_FILE_ECG_TRACK_DATA + MPT_FILE_TRACK_ENDING + MPT_FILE_TRACK_HEADER) + MPT_FILE_ECG_TRACK_DATA
                    track           = binaryFile[trackInitialPos:trackFinalPos]
                    
                    # Get the track code for its equivalence (format is short little endian)
                    trackCodes.append(struct.unpack('<h', binaryFile[pointLocations[i] + FILE_HEADER_TOTAL_SIZE + MPT_FILES_COMMON_HEADER_SIZE + MPT_FILE_TRACK_HEADER_TRACK_NUMBER_1 + j*MPT_FILE_TOTAL_SIZE:pointLocations[i] + FILE_HEADER_TOTAL_SIZE + MPT_FILES_COMMON_HEADER_SIZE + MPT_FILE_TRACK_HEADER_TRACK_NUMBER_2 + j*MPT_FILE_TOTAL_SIZE+1])[0])
                    
                    # Each track is encoded as 2500 *little endian* signed short values
                    trackFormat     = ['h' for k in range(0,MPT_FILE_ECG_TRACK_DATA,2)]
                    trackFormat     = '<' + ''.join(trackFormat)
                    track           = struct.unpack(trackFormat,track)
                    
                    # Store the track into the matrix
                    ekgData[j,:]    = track
                    
                    # Calculate LAT for each track
                    LAT = R_WAVE_POSITION - LAT_CALCULATION_WINDOW - 1
                    
                    for k in range(R_WAVE_POSITION - LAT_CALCULATION_WINDOW - 1, R_WAVE_POSITION + LAT_CALCULATION_WINDOW):
                        if track[k] > track[LAT]:
                            LAT = k
                    
                    # Introduce the LAT value for the track
                    latData[j,:] = (LAT - (R_WAVE_POSITION - 1))
                
                # Retrieve the information of the position of the catheter for this point
                catheter1InfoStartingPosition = pointLocations[i] + FILE_HEADER_TOTAL_SIZE + MPT_FILES_COMMON_HEADER_SIZE + MPT_FILE_TOTAL_SIZE*numberOfTracks[0] + CATHETER_POSITION_HEADER
                catheter2InfoStartingPosition = pointLocations[i] + FILE_HEADER_TOTAL_SIZE + MPT_FILES_COMMON_HEADER_SIZE + MPT_FILE_TOTAL_SIZE*numberOfTracks[0] + CATHETER_POSITION_HEADER + CATHETER_POSITION_TRACK_SIZE + CATHETER_POSITION_HEADER

                for section in range(0, CATHETER_SEGMENTS):
                    # The (x,y,z,alpha,beta,gamma) information is decoded as little endian floats
                    xPositionCatheter1 = struct.unpack('<f',binaryFile[catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section:catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4])
                    yPositionCatheter1 = struct.unpack('<f',binaryFile[catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION:catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION])
                    zPositionCatheter1 = struct.unpack('<f',binaryFile[catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION:catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION])
                    alphaCatheter1     = struct.unpack('<f',binaryFile[catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION+Z_POSITION:catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION+Z_POSITION])
                    betaCatheter1      = struct.unpack('<f',binaryFile[catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA:catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA])
                    gammaCatheter1     = struct.unpack('<f',binaryFile[catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA+ANGLE_BETA:catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA+ANGLE_BETA])

                    xPositionCatheter2 = struct.unpack('<f',binaryFile[catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section:catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4])
                    yPositionCatheter2 = struct.unpack('<f',binaryFile[catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION:catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION])
                    zPositionCatheter2 = struct.unpack('<f',binaryFile[catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION:catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION])
                    alphaCatheter2     = struct.unpack('<f',binaryFile[catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION+Z_POSITION:catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION+Z_POSITION])
                    betaCatheter2      = struct.unpack('<f',binaryFile[catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA:catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA])
                    gammaCatheter2     = struct.unpack('<f',binaryFile[catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA+ANGLE_BETA:catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA+ANGLE_BETA])
                    
                    # Save the information into the data matrix
                    catheter1Data[0,section] = xPositionCatheter1[0]
                    catheter1Data[1,section] = yPositionCatheter1[0]
                    catheter1Data[2,section] = zPositionCatheter1[0]
                    catheter1Data[3,section] = alphaCatheter1[0]
                    catheter1Data[4,section] = betaCatheter1[0]
                    catheter1Data[5,section] = gammaCatheter1[0]
                    
                    catheter2Data[0,section] = xPositionCatheter2[0]
                    catheter2Data[1,section] = yPositionCatheter2[0]
                    catheter2Data[2,section] = zPositionCatheter2[0]
                    catheter2Data[3,section] = alphaCatheter2[0]
                    catheter2Data[4,section] = betaCatheter2[0]
                    catheter2Data[5,section] = gammaCatheter2[0]
                    
                
                # Write the point data into files
                if not os.path.exists(os.path.join(rootdir,'OUTPUT') + subdir.replace(rootdir,'')):
                    os.makedirs(os.path.join(rootdir,'OUTPUT') + subdir.replace(rootdir,''))

                outputFileNameCatheter1 = os.path.join(os.path.join(rootdir,'OUTPUT') + subdir.replace(rootdir,''), os.path.splitext(file)[0] + '_P' + str(pointNumbers[i]) + '_Catheter1' + '.txt')
                outputFileNameCatheter2 = os.path.join(os.path.join(rootdir,'OUTPUT') + subdir.replace(rootdir,''), os.path.splitext(file)[0] + '_P' + str(pointNumbers[i]) + '_Catheter2' + '.txt')
                outputFileNameEKG       = os.path.join(os.path.join(rootdir,'OUTPUT') + subdir.replace(rootdir,''), os.path.splitext(file)[0] + '_P' + str(pointNumbers[i]) + '_EKG' + '.txt')
                outputFileNameLAT       = os.path.join(os.path.join(rootdir,'OUTPUT') + subdir.replace(rootdir,''), os.path.splitext(file)[0] + '_P' + str(pointNumbers[i]) + '_LAT' + '.txt')
                
                # Generate the header for the track files
                headerTracks        = '\t'.join([TRACK_EQUIVALENCE[j] for j in trackCodes]) + '\n'
                
                # Generate the header for the catheter files
                headerCatheter      = 'X\tY\t\Z\tAlpha\tBeta\tGamma\n'
                
                # Open files to write
                outputFileCatheter1 = open(outputFileNameCatheter1, "w")
                outputFileCatheter2 = open(outputFileNameCatheter2, "w")
                outputFileEKG       = open(outputFileNameEKG, "w")
                outputFileLAT       = open(outputFileNameLAT, "w")
                
                # Write headers
                outputFileEKG.write(headerTracks)
                outputFileLAT.write(headerTracks)
                outputFileCatheter1.write(headerCatheter)
                outputFileCatheter2.write(headerCatheter)

                # Write information into track-based files
                for j in range(0,ekgData.shape[1]):
                    outputFileEKG.write('\t'.join([str(ekgData[k,j]) for k in range(0,ekgData.shape[0])]) + '\n')

                outputFileLAT.write('\t'.join([str(latData[k,0]) for k in range(0,latData.shape[0])]) + '\n')
                
                # Write information into catheter-based files
                for j in range(0,catheter1Data.shape[1]):
                    outputFileCatheter1.write('\t'.join([str(catheter1Data[k,j]) for k in range(0,catheter1Data.shape[0])]) + '\n')
                    outputFileCatheter2.write('\t'.join([str(catheter2Data[k,j]) for k in range(0,catheter2Data.shape[0])]) + '\n')
                
                outputFileCatheter1.close()
                outputFileCatheter2.close()
                outputFileEKG.close()
                outputFileLAT.close()
                
            print(" * " + os.path.join(subdir.replace(rootdir,'')[1:],file))
                    
    

 * 20_NEUS/CARTO/DICOMDIR.dicm
 * 20_NEUS/CARTO/M0001_1-Refernencia.dicm
 * 20_NEUS/CARTO/M0002_2-Epi_Basal.dicm
 * 20_NEUS/CARTO/M0003_3-endo_basal.dicm
 * 20_NEUS/CARTO/M0004_4-Aorta.dicm
 * 20_NEUS/CARTO/M0005_5-endo_bri.dicm
 * 20_NEUS/CARTO/M0006_6-Epi_BRI.dicm
 * 20_NEUS/CARTO/M0007_7-Endo_VV0.dicm
 * 20_NEUS/CARTO/M0008_8-Epi_VV0.dicm
 * 20_NEUS/CARTO/S0009_0001.dicm
 * 02_ADA/CARTO/DICOMDIR.dicm
 * 02_ADA/CARTO/M0001_1-Map.dicm
 * 02_ADA/CARTO/M0002_2-Map.dicm
 * 02_ADA/CARTO/M0003_3-Map.dicm
 * 02_ADA/CARTO/M0004_4-Map.dicm
 * 02_ADA/CARTO/M0005_5-VI_endo_Normal.dicm
 * 02_ADA/CARTO/M0006_6-VI_endo_post_bloqueo.dicm
 * 02_ADA/CARTO/M0007_7-Map.dicm
 * 02_ADA/CARTO/M0008_8-Epicardio_post_bloqueo.dicm
 * 02_ADA/CARTO/S0009_0001.dicm
 * 03_SILVIA/CARTO/DICOMDIR.dicm
 * 03_SILVIA/CARTO/M0001_1-aorta.dicm
 * 03_SILVIA/CARTO/M0002_2-VI.dicm
 * 03_SILVIA/CARTO/M0003_3-Map.dicm
 * 03_SILVIA/CARTO/M0004_4-VI_post_bloquieo_completo.dicm
 * 03_SILVIA/CARTO/S0005_0001.dicm
 * 04_JOANA/CAR

 * 15_GIZEM/CARTO/MIXED_15_16/M0017_1-CARTOSOUND.dicm
 * 15_GIZEM/CARTO/MIXED_15_16/S0010_0001.dicm
 * 15_GIZEM/CARTO/MIXED_15_16/S0022_0001.dicm
 * 15_GIZEM/CARTO/MIXED_15_16/U0018_0001.dicm
 * 15_GIZEM/CARTO/MIXED_15_16/U0019_0002.dicm
 * 15_GIZEM/CARTO/MIXED_15_16/U0020_0003.dicm
 * 15_GIZEM/CARTO/MIXED_15_16/U0021_0004.dicm
 * 16_ANNA/CARTO/MIXED_15_16/C0012_Study1.44.192.5.dicm
 * 16_ANNA/CARTO/MIXED_15_16/C0016_Study1.45.192.5.dicm
 * 16_ANNA/CARTO/MIXED_15_16/DICOMDIR.dicm
 * 16_ANNA/CARTO/MIXED_15_16/I0011_Default.dicm
 * 16_ANNA/CARTO/MIXED_15_16/I0013_Default.dicm
 * 16_ANNA/CARTO/MIXED_15_16/I0014_LV.dicm
 * 16_ANNA/CARTO/MIXED_15_16/I0015_CS.dicm
 * 16_ANNA/CARTO/MIXED_15_16/M0001_1-prove.dicm
 * 16_ANNA/CARTO/MIXED_15_16/M0002_2-Epi basal.dicm
 * 16_ANNA/CARTO/MIXED_15_16/M0003_3-Endo basal.dicm
 * 16_ANNA/CARTO/MIXED_15_16/M0004_4-aorta.dicm
 * 16_ANNA/CARTO/MIXED_15_16/M0005_5-aplicaciones.dicm
 * 16_ANNA/CARTO/MIXED_15_16/M0006_6-Endo Post.dicm
 * 16_ANNA/CARTO/MIXED_15

 * 31_CRIS/CARTO/MIXED_30_31/M0013_2-EPI_BASAL.dicm
 * 31_CRIS/CARTO/MIXED_30_31/M0014_3-ENDO_BASAL.dicm
 * 31_CRIS/CARTO/MIXED_30_31/M0015_4-ENDO_RVPACING.dicm
 * 31_CRIS/CARTO/MIXED_30_31/M0016_5-EPI_RVPACING.dicm
 * 31_CRIS/CARTO/MIXED_30_31/M0017_6-ENDO_BIV.dicm
 * 31_CRIS/CARTO/MIXED_30_31/M0018_7-EPI_BIV.dicm
 * 31_CRIS/CARTO/MIXED_30_31/S0011_0001.dicm
 * 31_CRIS/CARTO/MIXED_30_31/S0019_0001.dicm
 * 32_KIRA/CARTO/DICOMDIR.dicm
 * 32_KIRA/CARTO/M0001_1-REF.dicm
 * 32_KIRA/CARTO/M0002_2-EPI_BASAL.dicm
 * 32_KIRA/CARTO/M0003_3-EPI_BASAL_2.dicm
 * 32_KIRA/CARTO/M0004_4-Map.dicm
 * 32_KIRA/CARTO/M0005_5-EPI_BASAL.dicm
 * 32_KIRA/CARTO/M0006_6-ENDO_BASAL.dicm
 * 32_KIRA/CARTO/M0007_7-AORTA.dicm
 * 32_KIRA/CARTO/M0008_8-AORTA_2.dicm
 * 32_KIRA/CARTO/M0009_9-ENDO_POST.dicm
 * 32_KIRA/CARTO/M0010_10-EPI_POST.dicm
 * 32_KIRA/CARTO/M0011_11-EPI_PACING.dicm
 * 32_KIRA/CARTO/M0012_12-ENDO_PACING.dicm
 * 32_KIRA/CARTO/M0013_13-prueba_qrs.dicm
 * 32_KIRA/CARTO/M0014_14-prueba_QRS_2.dicm
 * 32_